In [5]:
# imports of the loaded dataframes
from data_loading import df_gender, df_judging_perceiving, df_extrovert_introvert, df_feeling_thinking

# import of needed libraries
import pandas as pd
import numpy as np
import seaborn as sns

In [7]:
df_gender 

,auhtor_ID,post,female
0,t2_rnjzutp,Good on you for being responsible! I know self...,1
1,t2_rnjzutp,"must go to the grocery store with their child,...",1
2,t2_rnjzutp,"things on her videos, and YouTube took the vid...",1
3,t2_rnjzutp,their app. There's also a program called SYNC ...,1
4,t2_rnjzutp,"side. If the cops don't take your side, you'll...",1
...,...,...,...
44630,t2_6mpla2l0,if smegma kept her kids away just out of spite...,1
44631,t2_6mpla2l0,PhDs to change the time on my microwave. I did...,1
44632,t2_6mpla2l0,HiLIARy could even think of doing! I think Car...,1
44633,t2_6mpla2l0,of the hand is a breeze. It swells after thoug...,1


In [8]:
df_extrovert_introvert

,auhtor_ID,post,extrovert
0,t2_2hrxxs28,"I have a question, if you have no doctor, how'...",0
1,t2_2hrxxs28,butt to have Covid + your cycle. I'm not sure ...,0
2,t2_2hrxxs28,through different doctors. My situation sucks ...,0
3,t2_4pxpgwz,i thought it was about the pebbleyeet guy the ...,0
4,t2_4pxpgwz,…i always end up voting wrong even as crewmate...,0
...,...,...,...
40447,t2_8xjvf,Professional Association** We do not allow per...,0
40448,t2_8xjvf,work on the artist's behalf. Please post this ...,0
40449,t2_8xjvf,* Chill * Alternative/Alt * LoFi * Electronic ...,0
40450,t2_8xjvf,without the ability to appeal. ___ ___ If you ...,0


In [9]:
df_judging_perceiving

,auhtor_ID,post,judging
0,t2_yo59v,"Oh...but why? Seriously, even corrected it's s...",0
1,t2_yo59v,could make out the address the second time and...,0
2,t2_yo59v,That's what he said yeah. But no he was genuin...,0
3,t2_yo59v,"here. I feel like he knows, with this wording ...",0
4,t2_yo59v,time it picked up steam lots of (ex)slaves got...,0
...,...,...,...
41360,t2_8xjvf,precarious mental health (and therefore percep...,1
41361,t2_8xjvf,"know, you can talk to other people about your ...",1
41362,t2_8xjvf,"of well-rated TV shows from 2015-16 onwards, a...",1
41363,t2_8xjvf,Biden story in particular - only that Facebook...,1


In [10]:
df_feeling_thinking

,auhtor_ID,post,feeling
0,t2_dzpdnx2b,"Bragging is ridiculous, but maybe some of the ...",0
1,t2_dzpdnx2b,"about this person, and I'm not entirely sure w...",0
2,t2_dzpdnx2b,"kindly, and just keep to myself, I really want...",0
3,t2_dzpdnx2b,"yourself and improve yourself, to be able to s...",0
4,t2_dzpdnx2b,"never, but I'm surrounded by intps and infps I...",0
...,...,...,...
39595,t2_bkl0y,African genes... then can you put African-Amer...,0
39596,t2_bkl0y,old houses. But the older the house the more l...,0
39597,t2_bkl0y,soccer places. Talk to anyone who might even b...,0
39598,t2_bkl0y,you can do something. Even in your living room...,0


In [12]:
from autocorrect import Speller
 
check = Speller(lang='en')

text = "i wanna see u agan"

text = check(text)

text

'i wanna see u again'

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Download WordNet data (needed for lemmatization) and punkt for normal functioning of tokinizer
nltk.download('wordnet')
nltk.download('punkt')

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    
    # Tokenize the text into words
    tokens = nltk.word_tokenize(text)
    
    # Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word, pos=wordnet.VERB) for word in tokens]
    
    # Join lemmatized words back into a string
    lemmatized_text = ' '.join(lemmatized_words)
    
    return lemmatized_text

# Example text
text = "The dogs are barking loudly outside."

# Lemmatize the text
lemmatized_text = lemmatize_text(text)
print(lemmatized_text)


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexraudvee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alexraudvee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The dog be bark loudly outside .


In [8]:
from functions import tokenizer, remove_stop_words, remove_upercase, remove_punctuation

import itertools

# Define your functions
def function_1(text):
    return text.upper()

def function_2(text):
    return text.lower()

# Define more functions (function_3 to function_16) in a similar manner

# Store your functions in a list
function_list = [remove_stop_words, remove_upercase, remove_punctuation]  # Add all your functions here

# Example text
input_text = "This is an example text."

# Apply functions in parallel
# Generate combinations of functions and apply them to the text
for r in range(1, len(function_list) + 1):
    for combination in itertools.combinations(function_list, r):
        processed_text = input_text
        for func in combination:
            processed_text = func(processed_text)
            
        if type(processed_text) is str:
            processed_text = tokenizer(processed_text)

        print(f"Processed text with {' -> '.join(f.__name__ for f in combination)}:", processed_text)




Processed text with remove_stop_words: ['example', 'text', '.']
Processed text with remove_upercase: ['this', 'is', 'an', 'example', 'text', '.']
Processed text with remove_punctuation: ['This', 'is', 'an', 'example', 'text']
Processed text with remove_stop_words -> remove_upercase: ['example', 'text', '.']
Processed text with remove_stop_words -> remove_punctuation: ['example', 'text']
Processed text with remove_upercase -> remove_punctuation: ['this', 'is', 'an', 'example', 'text']
Processed text with remove_stop_words -> remove_upercase -> remove_punctuation: ['example', 'text']


Original Sentence: this is an example sentence .
Bigrams: this_is an example sentence .
Original Sentence: it demonstrates how to generate multi-word phrases using gensim library .
Bigrams: it demonstrates how to generate multi-word phrases using gensim library .
Original Sentence: this is another sentence for testing .
Bigrams: this_is another sentence for testing .
